In [28]:
from mnist_loader import load_data_wrapper
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import time

In [29]:
dane_treningowe, dane_walidacyjne, dane_testowe = load_data_wrapper()

In [ ]:
dane_treningowe = list(dane_treningowe)
dane_walidacyjne = list(dane_walidacyjne)
dane_testowe = list(dane_testowe)

print(len(dane_treningowe))
print(len(dane_walidacyjne))
print(len(dane_testowe))

50000
10000
10000


#### Definiujemy byty

* Warstwa, metody:
1. forward propagation
2. back propagation

 Warstwa powinna mieć atrybuty:
- wagi
- dane_na_wejsciu (zapamietać)
- dane_na_wyjsciu (przekazujemy)
- liczba neuronow
- funkcja aktywacji
- net (zapamiętać)

Zapamiętać np. w liście

* Sieć (składa sie z wielu Warstw), metody
1. fit
2. predict
3. dokladnosc


In [4]:
def wygeneruj_wagi(wymiar_wejscie, wymiar_wyjscie):
    # wyjściem jest liczba neuronów w następnej warstwie
    wektor_wag = np.random.normal(0,1/math.sqrt(wymiar_wejscie),(1+wymiar_wejscie)*wymiar_wyjscie)
    return np.reshape(wektor_wag, (wymiar_wyjscie, 1+wymiar_wejscie )  )

# Definiujemy funkcje aktywacji fi (sigmoid)
# fi' = fi*(1-fi) 
def sigmoid(x): return (1+np.exp(-x))**(-1) 

def deriv_sigmoid(x): return sigmoid(x)*(1-sigmoid(x))

In [ ]:
class SiecNeuronowa:
    def __init__(self, wymiary = [784, 128, 64, 10], epoki = 10):
        self.warstwa = Warstwa()

    def forward_propagation(self):
        pass

    def backward_propagation(self):
        pass
        
    def fit(self):
        pass

    def predict(self):
        pass

    def dokladnosc(self):
        pass

class Warstwa:
    def __init__(self, liczba_neuronow, dane_wejscie):
        """liczba neuronów - w warstwie (czyli liczba danych na wyjściu)"""
        self.liczba_neuronow = liczba_neuronow

        self.dane_wejscie = dane_wejscie
        self.liczba_wejscie = len(self.dane_wejscie)
        # Ustalamy stałą uczenia dla całej warstwy
        self.stala_uczenia = 0.1

        # Inicjalizacja wag
        self.wagi = wygeneruj_wagi(self.liczba_neuronow)

        # Rzeczy do zapamiętania
        self.lista_dane_wejscie = list()
        self.lista_net = list()
    

    def fun_aktywacji(self, x): return sigmoid(x)

    def pochodna_fun_aktywacji(self, x): return deriv_sigmoid(x)

    def forward_prop(self, dane_wejscie ):
        """Definiujemy propagację w przód.
        Podajemy dane na wejście, dodajemy na bias = 1 na początek wektora (otrzymujemy w ten sposób X).
        Zadajemy najpierw ile chcemy mieć neuronów na wyjściu
        Nastepnie liczymy net = W*X <- wyjście netto.
        Póżniej nakładamy funkcję aktywacji fi na net, otrzymując fi(net) = a <- wyjście z warstwy.
        Musimy też zapisywać stany x, net, a dla każdej warstwy (np. listy)"""

        # Musimy przerobić X na wektor kolumnowy
        self.X = np.vstack([1, dane_wejscie.reshape(-1,1)])
        self.net = self.wagi @ self.X
        dane_wyjscie = self.fun_aktywacji(self.net)

        self.lista_net.append(self.net)
        self.lista_dane_wejscie.append(self.X)

        self.wyjscia_forward_prop = dane_wyjscie

        return dane_wyjscie
    
    def back_prop(self, wyjscie_oczekiwane):
        """Definiujemy propagację wstecz.
        W ostatniej warstwie liczymy funkcję straty, następnie pochodną dL/da."""
        
        # Funkcja straty to L = 1/2(a[L]-y)^2, czyli pochodna z L to a[L]-y
        dL_a = self.wyjscia_forward_prop - wyjscie_oczekiwane

        # mamy pochodną dL/da * fi(net)
        delta = dL_a * self.pochodna_fun_aktywacji( self.net )
        dL_dW = delta @ self.X.T

        # Liczymy dL/dX i usuwamy pierwszy element, otrzymując dL/da niższej warstwy
        dL_dX = self.wagi.T @ delta
        
        # Dane do przekazania warstwę niżej
        dL_a = dL_dX[1:]

        # Aktualizujemy wagi
        self.wagi = self.wagi - self.stala_uczenia * dL_dW

        return dL_a

